# Importação e preparação dos dados

In [2]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_enunciados.csv', sep = '|')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_ENUNCIADO,TEXTO_ENUNCIADO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995437,Não comprovada a participação do beneficiário ...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025587,A operação ficta de exportação de plataformas ...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455370,"No âmbito do TCU, a dosimetria da pena tem com...",Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773746,O princípio da independência das instâncias pe...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773402,É vedado aos dirigentes das entidades do Siste...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13312, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          557
Contrato Administrativo     942
Convênio                    685
Desestatização              140
Direito Processual         1813
Finanças Públicas           328
Gestão Administrativa       339
Licitação                  2765
Pessoal                    3396
Responsabilidade           2347
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13312, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 200
dim_vetor = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_ENUNCIADO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_ENUNCIADO'])

Using TensorFlow backend.


Found 15387 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13312, 200)


In [9]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('model.txt')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, 100))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 15387
Encontrados no modelo: 13758 = 89.41314096315071


# Treinamento

## Embedding com pesos fixos

In [11]:
from keras.layers import Embedding

embedding = Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False)

W0814 20:28:36.535892 139766008571712 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [12]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(embedding)
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

W0814 20:28:36.925387 139766008571712 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 20:28:37.014918 139766008571712 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0814 20:28:37.066270 139766008571712 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0814 20:28:37.067885 139766008571712 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              40962048  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10250     
Total para

In [13]:
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0814 20:28:49.309681 139766008571712 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 20s 2ms/step - loss: 1.8962 - categorical_accuracy: 0.3776 - val_loss: 1.5626 - val_categorical_accuracy: 0.4071
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 1.3631 - categorical_accuracy: 0.5450 - val_loss: 1.3286 - val_categorical_accuracy: 0.5700
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 1.1084 - categorical_accuracy: 0.6336 - val_loss: 1.2555 - val_categorical_accuracy: 0.5922
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.9091 - categorical_accuracy: 0.6951 - val_loss: 1.3502 - val_categorical_accuracy: 0.5621
Epoch 5/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.7568 - categorical_accuracy: 0.7465 - val_loss: 1.3908 - val_categorical_accuracy: 0.5659
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.6238 - categori

In [14]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 18s 2ms/step - loss: 1.6883 - categorical_accuracy: 0.4380 - val_loss: 1.4044 - val_categorical_accuracy: 0.5415
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 1.0803 - categorical_accuracy: 0.6397 - val_loss: 1.3222 - val_categorical_accuracy: 0.5648
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.7369 - categorical_accuracy: 0.7593 - val_loss: 1.4015 - val_categorical_accuracy: 0.5468
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.4571 - categorical_accuracy: 0.8467 - val_loss: 1.7058 - val_categorical_accuracy: 0.5276
Epoch 5/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.2789 - categorical_accuracy: 0.9058 - val_loss: 1.6914 - val_categorical_accuracy: 0.5896
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.1889 - categori

In [15]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 17s 2ms/step - loss: 1.7516 - categorical_accuracy: 0.4160 - val_loss: 1.5190 - val_categorical_accuracy: 0.4720
Epoch 2/20
10649/10649 [==============================] - 16s 2ms/step - loss: 1.1711 - categorical_accuracy: 0.6075 - val_loss: 1.3166 - val_categorical_accuracy: 0.5543
Epoch 3/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.8482 - categorical_accuracy: 0.7139 - val_loss: 1.3991 - val_categorical_accuracy: 0.5670
Epoch 4/20
10649/10649 [==============================] - 16s 1ms/step - loss: 0.5888 - categorical_accuracy: 0.8031 - val_loss: 1.5990 - val_categorical_accuracy: 0.5633
Epoch 5/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.3672 - categorical_accuracy: 0.8786 - val_loss: 1.7246 - val_categorical_accuracy: 0.5956
Epoch 6/20
10649/10649 [==============================] - 16s 1ms/step - loss: 0.2324 - categori

## Embedding com pesos ajustáveis

In [17]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0814 20:46:08.714123 139766008571712 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
flatten_5 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 2048)              40962048  
_________________________________________________________________
dropout_9 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_10 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                10250     
Total para

In [18]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 19s 2ms/step - loss: 1.6529 - categorical_accuracy: 0.4502 - val_loss: 1.4642 - val_categorical_accuracy: 0.4976
Epoch 2/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.9489 - categorical_accuracy: 0.6814 - val_loss: 1.1560 - val_categorical_accuracy: 0.6297
Epoch 3/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.5632 - categorical_accuracy: 0.8128 - val_loss: 1.3088 - val_categorical_accuracy: 0.6061
Epoch 4/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.3006 - categorical_accuracy: 0.8989 - val_loss: 1.4459 - val_categorical_accuracy: 0.6312
Epoch 5/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.1601 - categorical_accuracy: 0.9462 - val_loss: 1.7332 - val_categorical_accuracy: 0.6632
Epoch 6/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.0996 - categori

In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 18s 2ms/step - loss: 1.7144 - categorical_accuracy: 0.4248 - val_loss: 1.3620 - val_categorical_accuracy: 0.5591
Epoch 2/20
10649/10649 [==============================] - 16s 2ms/step - loss: 1.0446 - categorical_accuracy: 0.6517 - val_loss: 1.1102 - val_categorical_accuracy: 0.6410
Epoch 3/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.6600 - categorical_accuracy: 0.7786 - val_loss: 1.1579 - val_categorical_accuracy: 0.6692
Epoch 4/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.3868 - categorical_accuracy: 0.8683 - val_loss: 1.5076 - val_categorical_accuracy: 0.6091
Epoch 5/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.2044 - categorical_accuracy: 0.9312 - val_loss: 1.5454 - val_categorical_accuracy: 0.6545
Epoch 6/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.1134 - categori